## INIT

In [ ]:
## For google colab only
from google.colab import drive
drive.mount('/content/drive')
import sys, os
## on Phuc's drive
%cd /content/drive/MyDrive/Uni/Thesis/Repo 
os.chdir("/content/drive/MyDrive/Uni/Thesis/Repo")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Uni/Thesis/Repo


In [ ]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 519 µs (started: 2022-11-26 07:12:54 +00:00)


In [ ]:
!pip install retrying

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 2.82 s (started: 2022-11-26 07:12:54 +00:00)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch, time, shutil, cv2, sys, os, json, torchvision
import torch.nn as nn
from skimage import io
import pandas as pd
from retrying import retry
from json import JSONEncoder
from pandas.io.json import json_normalize
from tqdm.notebook import tqdm
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from albumentations.pytorch.transforms import ToTensorV2
from torchvision import utils
from albumentations import (HorizontalFlip, ShiftScaleRotate, VerticalFlip, Normalize,Flip,
                            Compose, GaussNoise)

time: 1.68 s (started: 2022-11-26 07:12:57 +00:00)


In [ ]:
""" PY FILES """
py_file_location = "/content/drive/MyDrive/Uni/Thesis/Repo/utils"
sys.path.append(os.path.abspath(py_file_location))

time: 812 µs (started: 2022-11-26 07:12:59 +00:00)


In [ ]:
# To avoid non-essential warnings 
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

time: 792 µs (started: 2022-11-26 07:12:59 +00:00)


In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version
# Install PyTorch
!pip install torch==1.12.0 torchvision --extra-index-url https://download.pytorch.org/whl/cu113
# Install MMCV
!pip install openmim
!mim install mmcv-full==1.6.0
# !rm -rf mmsegmentation
# !git clone https://github.com/open-mmlab/mmsegmentation.git 
%cd mmsegmentation
!pip install -e .

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
/content/drive/MyDrive/Uni/Thesis/Repo/mmsegmentation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev

In [ ]:
from parse_dataset import *
from eval import *
from image_extract import *

load checkpoint from local path: /content/drive/MyDrive/Uni/Thesis/Repo/checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth
time: 4.6 s (started: 2022-11-26 07:13:24 +00:00)


## Loging setup

In [ ]:
!pwd

/content/drive/MyDrive/Uni/Thesis/Repo/mmsegmentation
time: 182 ms (started: 2022-11-26 07:13:46 +00:00)


In [ ]:
# Timestamp tag for check point and best model
from datetime import datetime, timezone
import pytz

def getCTimeStr():
  py_timezone = pytz.timezone('Australia/Adelaide')
  dt_string = datetime.now(py_timezone).strftime("(%H:%M:%S-%b_%d_%Y)")
  # print("date and time =", dt_string)	
  return str(dt_string)

time: 1.26 ms (started: 2022-11-26 07:13:47 +00:00)


In [ ]:
getCTimeStr()

'(17:43:47-Nov_26_2022)'

time: 23.5 ms (started: 2022-11-26 07:13:47 +00:00)


In [ ]:
# log dir - create/ check file and paste link below
LOG_DIR = '/content/drive/MyDrive/Uni/Thesis/Repo/log/tree_related_M.log'

time: 372 µs (started: 2022-11-26 07:13:47 +00:00)


In [ ]:
# Generate logger
import logging, time, os

# Create and configure logger
logging.basicConfig(filename=LOG_DIR,filemode='w') 

# Creating an object
logger = logging.getLogger()
 
# Setting the threshold of logger to DEBUG
logger.setLevel(logging.INFO)

# đổi repo link chỗ này
fh = logging.FileHandler(LOG_DIR)
# đổi thresh level - DEBUG < INFO < WARNING < ERROR < CRITICAL 
fh.setLevel(logging.INFO)
logger.addHandler(fh)

time: 3.25 ms (started: 2022-11-26 07:13:47 +00:00)


In [ ]:
# logger.info(f'|INFO| {getCTimeStr()}: init')

time: 267 µs (started: 2022-11-26 07:13:47 +00:00)


## Get 1000 trees list

(one time run only)

In [ ]:
from typing_extensions import dataclass_transform
dtecDS_info = "/content/drive/MyDrive/Uni/Thesis/dataset/detection/PasadenaUrbanTrees/detection_datasets.json"
with open(dtecDS_info) as json_data:
  data = json.load(json_data)

time: 1.85 s (started: 2022-11-21 12:27:54 +00:00)


In [ ]:
def getDtecDFInfo(ds):
    dsX = pd.DataFrame(list(ds['extended']))
    df = pd.concat([ds,dsX], axis=1)
    # df = df.drop(['Longitude','Latitude','extended'], axis=1)

    return df
dtecDF_Itrain, dtecDF_Itest, dtecDF_Ival = [getDtecDFInfo(pd.DataFrame(data[x])) for x in ('train', 'test', 'val')]
treeSamplesF = dtecDF_Itrain[['roi','OBJECTID','lat', 'lng']]


time: 169 ms (started: 2022-11-21 12:27:56 +00:00)


In [ ]:
len(treeSamplesF)

18185

time: 3.15 ms (started: 2022-11-21 12:13:53 +00:00)


In [ ]:
treeSamplesF

,roi,OBJECTID,lat,lng
0,ROI10,1019,34.146522,-118.147762
1,ROI10,1020,34.146634,-118.147764
2,ROI10,1021,34.146562,-118.147549
3,ROI10,1024,34.147326,-118.147779
4,ROI10,1025,34.147364,-118.147881
...,...,...,...,...
18180,ROI11,76749,34.130466,-118.138332
18181,ROI11,76750,34.133119,-118.138208
18182,ROI12,76751,34.145774,-118.135513
18183,ROI12,76752,34.145780,-118.135432


time: 19.3 ms (started: 2022-11-21 12:27:56 +00:00)


In [ ]:
data_np =  treeSamplesF.to_numpy()

time: 4.28 ms (started: 2022-11-21 12:27:56 +00:00)


In [ ]:
dsDir = '/content/drive/MyDrive/Uni/Thesis/dataset/detection/PasadenaUrbanTrees'
pDs = ParseDataset(dsDir)
panoL,camLatL, camLngL, distanceL = [], [], [], []

for row in data_np:
  lat, lng = row[2:4]
  temp_pano = pDs.streetview_extractor.get_nearest_pano(lat,lng)['Location']
  panoL.append(temp_pano['panoId'])
  camLatL.append(temp_pano['original_lat'])
  camLngL.append(temp_pano['original_lng'])
  distanceL.append(haversine_distance(float(lat),float(lng),float(temp_pano['original_lat']),float(temp_pano['original_lng'])))
  logger.info(f'|INFO| {getCTimeStr()}: Set Pano_ {temp_pano["panoId"]} for ({lat},{lng})')

Streaming output truncated to the last 5000 lines.
INFO:root:|INFO| (23:22:32-Nov_21_2022): Set Pano_ lRnTOSDcl8LJJ53jcLGuYw for (34.15336124984749,-118.12672037596371)
INFO:root:|INFO| (23:22:32-Nov_21_2022): Set Pano_ 9EcGasAf4ol4OXhIyu6nvg for (34.15705271138527,-118.12672199492208)
INFO:root:|INFO| (23:22:32-Nov_21_2022): Set Pano_ UHi8ZaWGlAJAkHWrjuGsRA for (34.15974065721811,-118.13106904684548)
INFO:root:|INFO| (23:22:32-Nov_21_2022): Set Pano_ p35hbI-3A6JpowR068cGAA for (34.15977479256878,-118.13107175088602)
INFO:root:|INFO| (23:22:32-Nov_21_2022): Set Pano_ tikLhFFr5_hYODJyPjhe0A for (34.160148681779795,-118.13106770004426)
INFO:root:|INFO| (23:22:33-Nov_21_2022): Set Pano_ xFyhJqhbjewK0CS60Fe-ig for (34.15569538555495,-118.12825297021766)
INFO:root:|INFO| (23:22:33-Nov_21_2022): Set Pano_ gyTY1LZuKUi3aOSQmPBsqw for (34.16258658766887,-118.12679208178973)
INFO:root:|INFO| (23:22:33-Nov_21_2022): Set Pano_ U5kp6cBKT4evA5Ev36w5BQ for (34.13140925433269,-118.14270580308263)
INFO

time: 33min 52s (started: 2022-11-21 12:27:56 +00:00)


In [ ]:
treeSamplesF['panoId'] = panoL
treeSamplesF['cam_lat'] = camLatL
treeSamplesF['cam_lng'] = camLngL
treeSamplesF['distance'] = distanceL
treeSamplesF[['pxl_x','pxl_y','pxl_h','height']] = None
treeSamplesF.reset_index()

,index,roi,OBJECTID,lat,lng,panoId,cam_lat,cam_lng,distance,pxl_x,pxl_y,pxl_h,height
0,0,ROI10,1019,34.146522,-118.147762,aAbBg_vCzltynugAdbU9ng,34.146575,-118.147713,7.429149,None,None,None,None
1,1,ROI10,1020,34.146634,-118.147764,aAbBg_vCzltynugAdbU9ng,34.146575,-118.147713,8.088577,None,None,None,None
2,2,ROI10,1021,34.146562,-118.147549,aAbBg_vCzltynugAdbU9ng,34.146575,-118.147713,15.134629,None,None,None,None
3,3,ROI10,1024,34.147326,-118.147779,zVggVtWlQZJtULdN9iJygQ,34.147288,-118.147623,15.015593,None,None,None,None
4,4,ROI10,1025,34.147364,-118.147881,lRVxFHML9yeYed3DIz3jWA,34.147379,-118.147621,24.011092,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18180,18180,ROI11,76749,34.130466,-118.138332,lYOJVblMhvInM-HyIkBycw,34.130442,-118.138263,6.936436,None,None,None,None
18181,18181,ROI11,76750,34.133119,-118.138208,x8Sj979NwK5Ct8Pma7qyZA,34.133105,-118.138292,7.879610,None,None,None,None
18182,18182,ROI12,76751,34.145774,-118.135513,avglkZDhpPSV9r_nROJ51A,34.145854,-118.135503,8.974126,None,None,None,None
18183,18183,ROI12,76752,34.145780,-118.135432,JSt1gc09YsE_w568oGutCA,34.145855,-118.135396,9.017370,None,None,None,None


time: 48.1 ms (started: 2022-11-21 13:01:48 +00:00)


In [ ]:
# Pano Json
pano_info = "/content/drive/MyDrive/Uni/Thesis/Repo/panos.json"
panoDict = dict()
with open(pano_info) as fp:
    panoDict = json.load(fp)

time: 45.4 s (started: 2022-11-21 10:19:41 +00:00)


In [ ]:
# treeSamples.iloc[0].OBJECTID

In [ ]:
# panoDict['aAbBg_vCzltynugAdbU9ng']['Location']['lat']

Appending row with relate pano image (only 1 deep level)

In [ ]:
# for idx in range(1000):
#     row = treeSamples.iloc[idx]
#     links = panoDict[row.panoId]['Links']
    
#     if len(links) != 0:
      
#         for pano in links:
#             data = row.copy()
#             data.panoId = pano['panoId']
#             data.cam_lat = panoDict[row.panoId]['Location']['original_lat']
#             data.cam_lng = panoDict[row.panoId]['Location']['original_lat']
#             data.distance= haversine_distance(float(data.lat), float(data.lng), float(temp_pano['original_lat']), float(temp_pano['original_lng']))
#             treeSamplesF = treeSamples.append(data, ignore_index=True)
#             logger.info(f'|INFO| {getCTimeStr()}: Object {row.OBJECTID} is added pano {pano["panoId"]}')

INFO:root:|INFO| (20:50:29-Nov_21_2022): Object 1019 is added pano 4tCsw3eu4VWZtOCzqRwfzg
INFO:root:|INFO| (20:50:29-Nov_21_2022): Object 1019 is added pano BAbPNv-VpsC3U8DBe_N4fA
INFO:root:|INFO| (20:50:29-Nov_21_2022): Object 1061 is added pano 1tc1RB0p1tWQ1jwBV4G7jQ
INFO:root:|INFO| (20:50:29-Nov_21_2022): Object 1061 is added pano AyhG3Ylv_yXEgrL4MoBSSQ
INFO:root:|INFO| (20:50:29-Nov_21_2022): Object 1176 is added pano 9eESG416fjLOy3X5FUvzgw
INFO:root:|INFO| (20:50:29-Nov_21_2022): Object 1176 is added pano dRNJMXQISG1UCT-G4UtehA
INFO:root:|INFO| (20:50:29-Nov_21_2022): Object 1226 is added pano 1cnnG9y0dI23um2ARvCKjw
INFO:root:|INFO| (20:50:29-Nov_21_2022): Object 1226 is added pano iDxtZJvYdRWEEQAMM5_c8Q
INFO:root:|INFO| (20:50:29-Nov_21_2022): Object 1277 is added pano kfRU2_ytut53OT4DV8tyVw
INFO:root:|INFO| (20:50:29-Nov_21_2022): Object 1277 is added pano _f5EVftf6jR-5jdxDZpnAw
INFO:root:|INFO| (20:50:29-Nov_21_2022): Object 1414 is added pano GAvPiFSGCFvzJvMmMWec-Q
INFO:root:

time: 42.4 s (started: 2022-11-21 10:20:29 +00:00)


In [ ]:
# treeSamplesF

,roi,OBJECTID,lat,lng,panoId,cam_lat,cam_lng,distance,pxl_x,pxl_y,pxl_h,height
0,ROI10,1019,34.146522,-118.147762,aAbBg_vCzltynugAdbU9ng,34.146575,-118.147713,7.429149,None,None,None,None
1,ROI7,1061,34.166295,-118.113202,r5qEjUL64l1pJMgsgMSC1Q,34.166247,-118.113098,11.014559,None,None,None,None
2,ROI7,1176,34.167332,-118.113183,GSAXSZ-j0YE7j9sqdSVbIg,34.167365,-118.113086,9.700171,None,None,None,None
3,ROI10,1226,34.142809,-118.147589,jQkcbtJa5divS1AQG1Albg,34.142789,-118.147535,5.437973,None,None,None,None
4,ROI5,1277,34.161289,-118.109506,GUa_WJvbtjsfVaS1hGSFiw,34.161293,-118.109437,6.324537,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3056,ROI12,68938,34.146024,-118.129948,zvo6Mp3alxwq3k4VLUNJPA,34.145900,34.145900,2113.652872,None,None,None,None
3057,ROI7,68990,34.161772,-118.112478,eC-nMVHvXYW8YgdkSOBEUw,34.161845,34.161845,317.651729,None,None,None,None
3058,ROI7,68990,34.161772,-118.112478,1n8ZeLy0XQu_93jtcVEQJQ,34.161845,34.161845,317.651729,None,None,None,None
3059,ROI5,69008,34.158984,-118.113039,ByAVkF2eWReMKmekvbAVAg,34.158966,34.158966,8.101370,None,None,None,None


time: 22.8 ms (started: 2022-11-21 10:21:11 +00:00)


In [ ]:
treeSamplesF.to_json('../tree_F_Samples.json',orient="records")

time: 56.1 ms (started: 2022-11-21 13:01:49 +00:00)


## Get extra view trees list

(one time run only)

In [ ]:
from typing_extensions import dataclass_transform
treeEData = "/content/drive/MyDrive/Uni/Thesis/Repo/tree_EView_Samples.json"
dfFull = pd.read_json(treeEData)

time: 1.6 s (started: 2022-11-26 07:13:56 +00:00)


In [ ]:
# Pano Json
pano_info = "/content/drive/MyDrive/Uni/Thesis/Repo/panos.json"
panoDict = dict()
with open(pano_info) as fp:
    panoDict = json.load(fp)

time: 49.3 s (started: 2022-11-26 07:13:58 +00:00)


In [ ]:
dfFull.columns.values

array(['roi', 'OBJECTID', 'lat', 'lng', 'panoId', 'cam_lat', 'cam_lng',
       'distance', 'pxl_x', 'pxl_y', 'pxl_h', 'height'], dtype=object)

time: 2.44 ms (started: 2022-11-26 07:14:47 +00:00)


In [ ]:
treeE2Lvl = pd.DataFrame(columns =dfFull.columns.values)

time: 2.88 ms (started: 2022-11-26 07:17:03 +00:00)


Appending row with relate pano image (only 1 deep level)

In [ ]:
for idx in range(len(dfFull)):
    row = dfFull.iloc[idx]
    try:  
      temp_pano = panoDict[row.panoId]['Location']
    except KeyError as e:
       logger.error(e)
       continue
    links = panoDict[row.panoId]['Links']
    
    if len(links) != 0:
      
        for pano in links:
            data = row.copy()
            data.panoId = pano['panoId']
            data.cam_lat = panoDict[row.panoId]['Location']['original_lat']
            data.cam_lng = panoDict[row.panoId]['Location']['original_lat']
            data.distance= haversine_distance(float(data.lat), float(data.lng), float(temp_pano['original_lat']), float(temp_pano['original_lng']))
            treeE2Lvl = treeE2Lvl.append(data, ignore_index=True)
            logger.info(f'|INFO| {getCTimeStr()}: Object {row.OBJECTID} is added pano {pano["panoId"]}')

Streaming output truncated to the last 5000 lines.
INFO:root:|INFO| (18:02:07-Nov_26_2022): Object 68106 is added pano Mk9gquVvNdcbzMycIZFKeg
INFO:root:|INFO| (18:02:07-Nov_26_2022): Object 68106 is added pano 9tupAJs0AXBizn4fWFAFPA
INFO:root:|INFO| (18:02:07-Nov_26_2022): Object 64984 is added pano d2OSJ5jrvYakWtPJ6AbDlA
INFO:root:|INFO| (18:02:07-Nov_26_2022): Object 64984 is added pano 6hV-ecOyb0HjjcUJ-frSjQ
INFO:root:|INFO| (18:02:07-Nov_26_2022): Object 64984 is added pano 6hV-ecOyb0HjjcUJ-frSjQ
INFO:root:|INFO| (18:02:07-Nov_26_2022): Object 64984 is added pano LHfw53o0526ZOs-JvAMctw
INFO:root:|INFO| (18:02:07-Nov_26_2022): Object 64984 is added pano H0EIKS_J2FgAAAQZHdJ57A
INFO:root:|INFO| (18:02:07-Nov_26_2022): Object 67477 is added pano gzWpQV4r3tcnJn9YV1mWBQ
INFO:root:|INFO| (18:02:07-Nov_26_2022): Object 67477 is added pano uIxw6yyeVXiqylww7PS7-w
INFO:root:|INFO| (18:02:07-Nov_26_2022): Object 67477 is added pano uIxw6yyeVXiqylww7PS7-w
INFO:root:|INFO| (18:02:08-Nov_26_2022)

time: 16min 23s (started: 2022-11-26 07:17:05 +00:00)


In [ ]:
treeE2Lvl

In [ ]:
pwd

In [ ]:
treeExtra.to_json('../tree_E2LView_Samples.json',orient="records")

## Tree height detection

In [ ]:
dsDir = '/content/drive/MyDrive/Uni/Thesis/dataset/detection/PasadenaUrbanTrees'
pDs = ParseDataset(dsDir)

# Pano Json
pano_info = "/content/drive/MyDrive/Uni/Thesis/Repo/panos.json"
panoDict = dict()
with open(pano_info) as fp:
    panoDict = json.load(fp)

treeE2L = pd.read_json('/content/drive/MyDrive/Uni/Thesis/Repo/tree_E2LView_Samples.json')

time: 43.6 s (started: 2022-11-22 14:12:29 +00:00)


In [ ]:
len(treeE2L)

37726

time: 3.49 ms (started: 2022-11-22 14:53:59 +00:00)


In [ ]:
for idx in range(len(treeE2L)):
    row = treeE2L.iloc[idx]
    try:
      pano = panoDict[row.panoId]
    except KeyError as e:
      logger.error(e)
      continue

    lat, lng = row.lat, row.lng

    ## Update pixel point
    pixelPt = pDs.streetview_extractor.geo_coords_to_streetview_pixel(pano,lat,lng)
    row.pxl_x, row.pxl_y = pixelPt

    ##update distance, pxl_h and physical height
    try:
        row.pxl_h, row.height = pDs.streetview_extractor.physical_height_cal(pano, lat, lng)
    except (IndexError,OSError,TimeoutError,FileNotFoundError) as e:
        logger.error(e)
        try:
            time.sleep(1)
            row.pxl_h, row.height = pDs.streetview_extractor.physical_height_cal(pano, lat, lng)
        except (IndexError,OSError,TimeoutError,FileNotFoundError) as e:
            logger.error(e)
            continue
    
    finally:
        treeExtra.iloc[idx] = row
        logger.info(f'|INFO| {getCTimeStr()}: Object {row.OBJECTID} height on pano_{row.panoId} is updated - {idx*100/len(treeE2L):.3f}%')

Streaming output truncated to the last 5000 lines.
INFO:root:|INFO| (10:34:30-Nov_27_2022): Object 46590 height on pano_ikswqFLjUHjmDNr8t03hnw is updated - 72.282%
INFO:root:|INFO| (10:34:31-Nov_27_2022): Object 46590 height on pano_X3fX9UC4DD-pdSnsHwrj8g is updated - 72.283%
INFO:root:|INFO| (10:34:32-Nov_27_2022): Object 46590 height on pano_f6NJ2XtaYKMnINfbQQ0xsQ is updated - 72.284%
INFO:root:|INFO| (10:34:33-Nov_27_2022): Object 46590 height on pano_9Xn-PLM3myA5M1zAzWe4iA is updated - 72.285%
INFO:root:|INFO| (10:34:34-Nov_27_2022): Object 46590 height on pano_ikswqFLjUHjmDNr8t03hnw is updated - 72.287%
INFO:root:|INFO| (10:34:35-Nov_27_2022): Object 46590 height on pano_c_k3sRgC6s57h8PAYJTtHg is updated - 72.288%
INFO:root:|INFO| (10:34:36-Nov_27_2022): Object 46590 height on pano_AptHOAymFUBbmy3pV7_cGg is updated - 72.289%
INFO:root:|INFO| (10:34:36-Nov_27_2022): Object 46590 height on pano_C6FH3dLouzOTUGa561YGmw is updated - 72.290%
INFO:root:|INFO| (10:34:37-Nov_27_2022): Obje

In [ ]:
treeExtra.iloc[2700]

roi                           ROI9
OBJECTID                      8193
lat                      34.165716
lng                    -118.140773
panoId      uzc9pPTaL2SqhjmCejnt0Q
cam_lat                   34.16576
cam_lng                   34.16576
distance                  5.446831
pxl_x                   731.461258
pxl_y                   475.435757
pxl_h                        174.0
height                   -7.140719
Name: 2700, dtype: object

time: 36.1 ms (started: 2022-11-22 23:50:59 +00:00)


In [ ]:
treeE2L.to_json('../tree_E2LView_Samples.json',orient="records")

In [ ]:
treeE2L = pd.read_json('/content/drive/MyDrive/Uni/Thesis/Repo/tree_E2LView_Samples.json')
treeE2L